<a href="https://colab.research.google.com/github/Aisha-Hagar/Hamoye_GenAI/blob/main/QuizD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
! pip install datasets
! pip install accelerate -U
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.

In [3]:
from datasets import load_dataset

In [4]:
raw_datasets = load_dataset('glue', 'mrpc')
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Write a code snippet that uses a tokenizer to process the sentence pairs from the 'train' split of the Microsoft Research Paraphrase Corpus (MRPC) dataset. The code should perform tokenization, enable padding, and truncation for the sentence pairs from sentence1 and sentence2. What is the number of tokens when you access the 3 content of the tokenized sentence1 after implementing padding and truncation?

In [7]:
from transformers import AutoTokenizer

In [8]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [45]:
tokenized_datasets = tokenizer(
    raw_datasets['train']['sentence1'],
    raw_datasets['train']['sentence2'],
    padding=True,
    truncation=True
)

In [46]:
len(tokenized_datasets['token_type_ids'][0])

103

In [20]:
def tokenize_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [21]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorWithPadding

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
samples = tokenized_datasets['train'][:]
samples = {k: v for k, v in samples.items() if k not in ['idx', 'sentence1','sentence2']}

In [30]:
max([len(i) for i in samples['input_ids']])

103

In [31]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([3668, 103]),
 'token_type_ids': torch.Size([3668, 103]),
 'attention_mask': torch.Size([3668, 103]),
 'labels': torch.Size([3668])}

Extracts and displays the first data sample from the 'test' split of the Microsoft Research Paraphrase Corpus (MRPC) dataset stored in the variable 'raw_test_dataset' obtained from the previously loaded dataset. What is the content of the first sentence?

In [32]:
raw_test_dataset = raw_datasets['test']
raw_test_dataset[0]

{'sentence1': "PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .",
 'sentence2': 'Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .',
 'label': 1,
 'idx': 0}

Preprocessing the dataset: Write a code snippet that uses a tokenizer to process the sentence pairs from the 'train' split of the Microsoft Research Paraphrase Corpus (MRPC) dataset. The code should perform tokenization for the sentence pairs from sentence1 and sentence2. What is the number of tokens when you access the 3 content of the tokenized sentence1?

In [52]:
tokenized_datasets = tokenizer(
    raw_datasets['train']['sentence1'],
    raw_datasets['train']['sentence2']
)

In [53]:
len(tokenized_datasets['token_type_ids'][0])

50

Loading data: What code snippet loads the Microsoft Research Paraphrase Corpus (MRPC) dataset using the Hugging Face `datasets` library within Python? Load it in preparation for the next questions

In [54]:
raw_datasets = load_dataset('glue', 'mrpc')

Extracts and displays the third data sample from the 'train' split of the Microsoft Research Paraphrase Corpus (MRPC) dataset stored in the variable 'raw_train_dataset' obtained from the previously loaded dataset. What is the content of the second sentence?

In [55]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[2]

{'sentence1': 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'sentence2': "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
 'label': 1,
 'idx': 2}


What are the correct features of the test dataset saved to the raw_test_variable earlier on?

In [56]:
raw_test_dataset = raw_datasets['test']
raw_test_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}